# TL - Contratos

In [3]:
import pandas as pd
import re

In [4]:
df = pd.read_excel(r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES\data\00_raw\secop\secop_obras_2019_2025.xlsx")
df = df[['nombre_entidad', 'nit_entidad', 'codigo_entidad', 'sector',
       'id_contrato', 'estado_contrato',
       'modalidad_de_contratacion', 'codigo_de_categoria_principal',
       'descripcion_del_proceso', 'fecha_de_inicio_del_contrato',
       'fecha_de_fin_del_contrato', 'tipodocproveedor', 'documento_proveedor',
       'codigo_proveedor', 'proveedor_adjudicado', 'origen_de_los_recursos',
       'destino_gasto', 'valor_del_contrato', 'c_digo_bpin', 'urlproceso',
       'presupuesto_general_de_la_nacion_pgn',
       'sistema_general_de_participaciones', 'sistema_general_de_regal_as',
       'recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
       'recursos_de_credito', 'recursos_propios', 'objeto_del_contrato',
       'anio']]
print(f"📍 DataFrame RAW:\n Dimensiones: {df.shape}")

📍 DataFrame RAW:
 Dimensiones: (8421, 28)


In [5]:
# FILTRO - Estado del contrato
ESTADOS_CORE = [
    "terminado",
    "modificado",
    "en ejecución",
    "cerrado"
]
df["estado_contrato"] = (df["estado_contrato"].str.strip().str.lower())
filter = df[df["estado_contrato"].isin(ESTADOS_CORE)].copy()

print(filter["estado_contrato"].value_counts())
print("Contratos core:", filter.shape[0])
print("Porcentaje retenido:", round(filter.shape[0] / df.shape[0] * 100, 2), "%")


estado_contrato
terminado       3093
modificado      2759
en ejecución    1023
cerrado          866
Name: count, dtype: int64
Contratos core: 7741
Porcentaje retenido: 91.92 %


In [6]:
# MAPEO - Familia UNSPSC
MAP_FAMILIA_UNSPSC = {
    "7210": "Mantenimiento y reparaciones",
    "7211": "Edificación residencial",
    "7212": "Edificación no residencial",
    "7214": "Infraestructura pesada",
    "7215": "Infraestructura especializada"
}
filter["familia_unspsc"] = (filter["codigo_de_categoria_principal"].str.extract(r"V1\.(72\d{2})"))
filter["familia_unspsc_desc"] = (filter["familia_unspsc"].map(MAP_FAMILIA_UNSPSC))


In [ ]:
def estandarizar_descripcion(texto):
    if pd.isna(texto):
        return texto
    
    texto = texto.lower()  # 1. minusculas
    
    # 2. eliminar signos de puntuacion basicos
    texto = re.sub(r'[.,;:]', '', texto)
    
    # 3. eliminar comillas y caracteres especiales comunes
    texto = re.sub(r'[\"“”\'’]', '', texto)
    
    # 4. eliminar caracteres no alfanumericos (excepto espacios)
    texto = re.sub(r'[^a-záéíóúñü0-9\s]', ' ', texto)
    
    # 5. normalizar espacios
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

filter['descripcion_proceso_std'] = filter['descripcion_del_proceso'].apply(estandarizar_descripcion)


In [8]:
filter['descripcion_proceso_std'].unique()

array(['mantenimiento rehabilitación de la carretera alterna al puerto de santa marta sector mamatoco terminal maritimo tramo 9007a incluidas estructuras de puentes vehiculares departamento magdalena',
       'mantenimiento rutinario de las vías a cargo del instituto nacional de vías dirección territorial cundinamarca código 40cn07 avenida longitudinal de occidente alo sector chusacá canoas pr0 0000 pr4 0500 canoas río bogotá pr4 0500 pr14 0000',
       'realizar las actividades de mantenimiento y reparación integral locativo preventivo y correctivo a todo costo incluido mano de obra suministros y repuestos a los muebles e inmuebles en las sedes de la unidad a nivel nacional las oficinas territoriales y los centros regionales y puntos de atenc',
       ...,
       'obras de adecuación mejoras y reparaciones locativas en el inmueble donde funciona la sede principal del ideam en la ciudad de bogotá',
       'rehabilitación y o recuperación y o construcción y o reconstrucción y o mantenim

In [9]:
filter["familia_unspsc"].value_counts()

familia_unspsc
7210    3633
7214    2375
7215    1014
7212     648
7211      71
Name: count, dtype: int64

________

## manteniemiento

In [ ]:
mantenimiento = filter[filter["familia_unspsc"] == '7210']
print(mantenimiento.shape)

(3633, 31)


In [ ]:
mant = mantenimiento.copy()


mant['descripcion_proceso_std'] = (
    mant['descripcion_proceso_std']
    .str.replace(r'(\w+)cion\b', r'\1ción', regex=True)
)

# Lista de palabras clave y sus variantes
palabras_clave = [
    "mantenimiento", "mantener",
    "suministro",
    "adecuación", "adecuar", "adecuaciones",
    "mejoramiento", "mejorar",
    "reparación", "reparar",
    "construcción", "construir",
    "modernización", "modernizar"
]

# Patrón: cualquier palabra clave y todo lo que sigue
patron = re.compile(r".*?\b(" + "|".join(map(re.escape, palabras_clave)) + r".*)", flags=re.IGNORECASE)

def cortar_o_mantener(texto):
    m = patron.match(texto)
    if m:
        return m.group(1).strip()  # devuelve desde la palabra clave
    else:
        return texto.strip()       # si no hay coincidencia, conserva el original

mant["descripcion_proceso_std"] = mant["descripcion_proceso_std"].apply(cortar_o_mantener)

# Frses largas
mant['descripcion_proceso_std'] = (
    mant['descripcion_proceso_std']
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

values = mant['descripcion_proceso_std'].value_counts()
values.to_excel('mant_2.xlsx')


_______________

In [12]:
pesada = filter[filter["familia_unspsc"] == '7214']
print(pesada.shape)
pes = pesada['descripcion_proceso_std'].value_counts()
pes.to_excel('pesada.xlsx')

(2375, 31)


In [13]:
especial = filter[filter["familia_unspsc"] == '7215']
print(especial.shape)
es = especial['descripcion_proceso_std'].value_counts()
es.to_excel('especial.xlsx')

(1014, 31)


__________________

In [14]:
filter["sector"] = (
    filter["sector"]
    .str.strip()
    .str.lower()
)

filter["familia_unspsc_desc"] = (
    filter["familia_unspsc_desc"]
    .str.strip()
)
tabla_familia_sector = (
    filter
    .groupby(["familia_unspsc_desc", "sector"])
    .size()
    .reset_index(name="n_contratos")
    .sort_values(["familia_unspsc_desc", "n_contratos"], ascending=[True, False])
)
tabla_familia_sector.head(15)
tabla_matriz = pd.pivot_table(
    filter,
    index="familia_unspsc_desc",
    columns="sector",
    values="id_contrato",
    aggfunc="count",
    fill_value=0
)

tabla_matriz

sector,agricultura,ambiente y desarrollo sostenible,cultura,defensa,deportes,educación nacional,hacienda y crédito público,inclusión social y reconciliación,industria,información estadística,...,no aplica/no pertenece,planeación,presidencia de la república,relaciones exteriores,salud y protección social,servicio público,tecnologías de la información y las comunicaciones,trabajo,transporte,"vivienda, ciudad y territorio"
familia_unspsc_desc,,,,,,,,,,,,,,,,,,,,,
Edificación no residencial,0,25,23,156,0,17,28,3,18,4,...,30,18,6,7,6,20,1,51,72,5
Edificación residencial,0,0,0,24,0,0,2,1,0,0,...,0,2,0,0,0,8,0,0,0,0
Infraestructura especializada,27,45,32,255,7,47,42,5,13,19,...,45,3,2,5,34,60,3,126,105,4
Infraestructura pesada,0,21,4,57,17,7,1,1,0,0,...,13,6,10,0,1,83,0,15,2112,0
Mantenimiento y reparaciones,55,54,55,1018,25,167,135,7,22,60,...,256,16,10,9,82,243,11,307,709,15


In [15]:
filter['c_digo_bpin'].value_counts().head(20)

c_digo_bpin
No Definido        6147
202300000000147     235
2017011000408        79
2018011000852        63
202300000000001      63
2018011000986        54
202300000000027      49
2018011001034        45
2018011000435        41
2019011000238        33
2018011001093        26
2018011000022        24
202300000000203      22
2018011000754        21
2018011000990        21
2018011000175        20
2018011001006        19
2018011000336        17
2018011000628        17
2018011000693        17
Name: count, dtype: int64

In [16]:
# DICCIONARIOS [MACRO CATEGORIAS]

SECTORES_TRANSPORTE = ["transporte", "defensa"]
SECTORES_SOCIAL = ["educación nacional", "salud y protección social",
                    "cultura", "deportes", "inclusión social y reconciliación"]
SECTORES_AMBIENTE = ["ambiente y desarrollo sostenible",
                     "agricultura", "minas y energía"]
SECTORES_URBANISMO = ["vivienda, ciudad y territorio",
                       "planeación", "interior"]
SECTORES_PRODUCTIVIDAD = ["industria", "trabajo", "servicio público"]

filter["sector"] = (filter["sector"].str.strip().str.lower())




____________________

In [17]:
filter['sector'].unique()

array(['transporte', 'inclusión social y reconciliación',
       'ambiente y desarrollo sostenible', 'ley de justicia', 'defensa',
       'hacienda y crédito público', 'servicio público',
       'salud y protección social',
       'tecnologías de la información y las comunicaciones', 'trabajo',
       'vivienda, ciudad y territorio', 'relaciones exteriores',
       'no aplica/no pertenece', 'cultura', 'minas y energía',
       'industria', 'información estadística', 'agricultura',
       'planeación', 'presidencia de la república', 'educación nacional',
       'interior', 'deportes'], dtype=object)

In [18]:
code = pd.read_excel(r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES\Codigo_72_contratos.xlsx")
print(code.shape)
code = code[code['Código segmento'] == 72]
print(code.shape)


(638, 8)
(416, 8)


In [19]:
code['Código Producto'].nunique()

416

In [20]:
7210: 'Servicios de mantenimiento y reparaciones de construcciones e instalaciones',
7211:   'Servicios de construcción de edificaciones residenciales',
7212:      'Servicios de construcción de edificaciones no residenciales',
7214: 'Servicios de construcción pesada',
7215:   'Servicios de mantenimiento y construcción de comercio especializado'



SyntaxError: illegal target for annotation (1565385194.py, line 1)

In [ ]:
code['Código Familia'].unique()

: 

In [ ]:
def resumen_unicos(df):
    """
    Imprime los valores únicos y su conteo para cada columna del DataFrame.
    """
    for col in df.columns:
        print(f"\n📌 Columna: {col}")
        print("-" * (10 + len(col)))
        
        # Conteo de valores únicos
        conteo = df[col].value_counts(dropna=False)
        
        # Imprimir resultados
        for valor, cantidad in conteo.items():
            print(f"{valor}: {cantidad}")


: 

: 